In [1]:
# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

In [3]:
import pyodbc 

conndw = pyodbc.connect('Driver={SQL Server};'
                      'Server=SAMET;'
                      'Database=Projet3;'
                      'Trusted_Connection=yes;')
cursorDW = conndw.cursor()

In [4]:
SQL_Query = pd.read_sql_query('''select DateKey,FullDate,Year,DayOfWeek FROM [dbo].[dim_timspan]''', conndw)

dim_timspan = pd.DataFrame(SQL_Query)

In [5]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimConditions]''', conndw)
DimConditions = pd.DataFrame(SQL_Query)

In [6]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimLocation]''', conndw)
DimLocation = pd.DataFrame(SQL_Query)

In [7]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimPolice]''', conndw)
DimPolice = pd.DataFrame(SQL_Query)

In [8]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimRoad]''', conndw)
DimRoad = pd.DataFrame(SQL_Query)

In [9]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimTypeAccidentSeverity]''', conndw)
DimTypeAccidentSeverity = pd.DataFrame(SQL_Query)

In [10]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[FactAccident]''', conndw)
FactAccident = pd.DataFrame(SQL_Query)

In [11]:
def decomposeefact(fact,li,li1):
    for i,j in enumerate(li):
        fact= pd.merge(fact,j,on=li1[i])
    return fact

In [12]:
listetable=[dim_timspan,DimConditions,DimLocation,DimPolice,DimRoad,DimTypeAccidentSeverity]
listeid=['DateKey','code_condition','code_location','code_police','code_road','code_severity']

In [13]:
FactAccident=decomposeefact(FactAccident,listetable,listeid)

In [14]:
FactAccident.shape

(1504150, 29)

In [15]:
###The idea in this section is to predict the severity of the accident according to the analyzed data.

In [16]:
FactAccident.head(5)

,Accident_Index,code_location,code_condition,code_police,code_road,DateKey,code_severity,Number_of_Vehicles,Number_of_Casualties,Did_Police_Officer_Attend_Scene_of_Accident,...,Longitude,Latitude,Urban_or_Rural_Area,Police_Force,Local_Authority_District,Local_Authority_Highway,Road_Type,Speed_limit,Accident_Severity,Nomenclature
0,200501BS00001,969956,752,24,25,20050104,2,1,1,1,...,-0.19117,51.4891,1,1,12,E09000020,Single carriageway,30,2,medium
1,200701BS70255,969328,602,24,25,20070516,2,3,1,1,...,-0.192366,51.4916,1,1,12,E09000020,Single carriageway,30,2,medium
2,200701BS70534,969328,602,24,25,20070917,2,1,1,1,...,-0.192366,51.4916,1,1,12,E09000020,Single carriageway,30,2,medium
3,200601BS70277,958601,602,24,25,20060529,2,2,1,1,...,-0.211712,51.52,1,1,12,E09000020,Single carriageway,30,2,medium
4,201201BS70113,958601,602,24,25,20120305,2,2,1,1,...,-0.211712,51.52,1,1,12,E09000020,Single carriageway,30,2,medium


In [17]:
# Required Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [18]:
#Setting up a dataframe with the data converted to int encoded values
#facilitating training
#FactAccident_class = FactAccident.copy()

FactAccident_class = FactAccident.iloc[100000:1500000,:]

var=['code_condition','Local_Authority_Highway','Number_of_Vehicles','Accident_Severity']
FactAccident_class=FactAccident_class[var]

le = LabelEncoder()

for i in FactAccident_class.columns:
    if i != 'Accident_Severity':
        FactAccident_class[i] = le.fit_transform(FactAccident_class[i])

In [19]:
FactAccident_class.head(10)

,code_condition,Local_Authority_Highway,Number_of_Vehicles,Accident_Severity
100000,591,148,0,2
100001,591,148,1,2
100002,591,148,0,2
100003,591,148,0,2
100004,591,148,0,2
100005,591,148,1,2
100006,591,148,1,2
100007,591,148,4,2
100008,591,148,1,2
100009,591,148,2,2


In [27]:
#Converting to array, separating class from attributes
Y = np.array(FactAccident_class['Accident_Severity'])
X = np.array(FactAccident_class.drop(columns=['Accident_Severity']))

In [28]:
#Creating test and training set, with 70% of data for training 0.3
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=42)

In [29]:
def treinarKNN(train_x, train_y, test_x, test_y, n_neighbors=1):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    return accuracy_score(pred_y, test_y), confusion_matrix(test_y, pred_y)

In [30]:
def treinarNaiveBayes(train_x, train_y, test_x, test_y):
    gnb = GaussianNB()
    gnb.fit(train_x, train_y)
    pred_y = gnb.predict(test_x)
    return accuracy_score(pred_y, test_y), confusion_matrix(test_y, pred_y)

In [33]:
#Training the KNN classification model, for k ranging from 1 to 11
print("==== Accuracy for each number of neighbors ====\n")
for k in range(1, 11):
    acc, cm = treinarKNN(train_x, train_y, test_x, test_y, k)
    print("k = %d: %.2f%%" % (k, 100*acc))
    print("Confusion matrix:")
    print(cm, end='\n\n')

==== Accuracy for each number of neighbors ====

k = 1: 84.20%
Confusion matrix:
[[   109    490   3991]
 [   522   3661  26933]
 [  4174  30266 349854]]

k = 2: 78.35%
Confusion matrix:
[[   208    918   3464]
 [   990   6536  23590]
 [  7687  54274 322333]]

k = 3: 89.52%
Confusion matrix:
[[    70    201   4319]
 [   398   1408  29310]
 [  1657   8118 374519]]

k = 4: 88.79%
Confusion matrix:
[[    11    327   4252]
 [    64   2301  28751]
 [   355  13353 370586]]

k = 5: 90.56%
Confusion matrix:
[[     9    170   4411]
 [    51   1011  30054]
 [   188   4774 379332]]

k = 6: 90.40%
Confusion matrix:
[[     4    167   4419]
 [    29   1261  29826]
 [    76   5824 378394]]

k = 7: 91.03%
Confusion matrix:
[[     0     87   4503]
 [    11    626  30479]
 [    32   2548 381714]]

k = 8: 90.91%
Confusion matrix:
[[     0     99   4491]
 [    10    794  30312]
 [    26   3227 381041]]

k = 9: 91.34%
Confusion matrix:
[[     0     53   4537]
 [     9    277  30830]
 [    23    917 383354]

In [31]:
#Training the Naive Bayes classification model
print("====Accuracy for the Naive Bayes algorithm ====")
acc, cm = treinarNaiveBayes(train_x, train_y, test_x, test_y)
print("%.2f%%" % ( 100*acc))
print("Confusion matrix:")
print(cm, end='\n\n')

====Accuracy for the Naive Bayes algorithm ====
91.25%
Confusion matrix:
[[     4     55   4531]
 [    32    244  30840]
 [   137   1151 383006]]



In [ ]:
## On voit que l'algorithme Naive Bayes a mieux performé dans la classification, et, de plus,
#il est possible de conclure qu'il est clairement possible de prédire la gravité d'un accident en fonction de 
#la situation au cas où il y aurait même un accident, encore plus avec une précision de 91.25 %